# Uniswap Sepolia Token Swap Demo
This notebook demonstrates how to:
1. Generate a new private key and get its public address.
2. Fund the address with testnet ETH (from another wallet).
3. Wrap ETH into WETH.
4. Approve Uniswap's SwapRouter.
5. Swap WETH for USDC.

We'll be using `web3.py` and the Uniswap V3 SwapRouter on Unichain Sepolia (Chain ID = 1301).

In [4]:
# 1. Imports and Environment Setup
import os
import secrets
from eth_account import Account
from web3 import Web3, HTTPProvider
from eth_account.signers.local import LocalAccount
from dotenv import load_dotenv

# Load environment variables (RPC URL etc.)
# Make sure you have a .env file with something like:
# WEB3_RPC_URL='https://<YOUR_UNICHAIN_SEPOLIA_URL>'
load_dotenv()

# We'll fetch the RPC URL from the environment.
RPC_URL = os.getenv("WEB3_RPC_URL")
if not RPC_URL:
    raise Exception("Please set WEB3_RPC_URL in your .env file.")

# Create a Web3 instance
w3 = Web3(HTTPProvider(RPC_URL))
print("Connected to:", w3.client_version)

Connected to: Geth/v1.101411.4-stable-efa05b1b/linux-amd64/go1.23.4


In [14]:
# 2. Generate a New Wallet
# We will generate a random private key in the classroom.
# This is purely for demo - do not use for real funds.

private_key = "0x" + secrets.token_hex(32)
account: LocalAccount = Account.from_key(private_key)

# TODO: add some info on how public keys are generated from private keys.

print("Generated a new wallet!")
print("Private key:", private_key)
print("Public address:", account.address)

Generated a new wallet!
Private key: 0x81a58d4cb38b530a6d152d58e3d5b5d9eac6badbf2df9084d4399ed39f11b46e
Public address: 0xa7450338B3c85eE8D08903e295805BE7aC6FE3EB


### 3. Fund the New Wallet
At this point, **pause** execution and send some testnet ETH
to the address shown above from another wallet or faucet.

*You can check the funding status on the block explorer:*  
https://sepolia.uniscan.xyz/


In [6]:
# 4. Confirm Wallet Balance
# Once you have sent some ETH, run this cell to confirm the balance.

balance = w3.eth.get_balance(account.address)
print("ETH Balance:", w3.from_wei(balance, "ether"), "ETH")
if balance == 0:
    print("No ETH balance yet. Please fund the wallet and try again.")
else:
    print("Balance detected! Proceed with the next steps.")

ETH Balance: 0.05 ETH
Balance detected! Proceed with the next steps.


### 5. Wrap ETH into WETH
The WETH contract on Unichain Sepolia is at:
`0x4200000000000000000000000000000000000006`.

To get WETH, we will call the `deposit()` function on the WETH
contract, passing ETH as `value` in the transaction.
This effectively "wraps" your ETH into an ERC20 token.

In [8]:
# 6. Execute the deposit transaction

WETH_ADDRESS = "0x4200000000000000000000000000000000000006"

# TODO: show what the `deposit()` function does in the WETH contract.

# Minimal ABI for the WETH contract: just deposit, withdraw, balanceOf, etc.
weth_abi = [
    {
        "constant": False,
        "inputs": [],
        "name": "deposit",
        "outputs": [],
        "stateMutability": "payable",
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name":"wad","type":"uint256"}],
        "name": "withdraw",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name":"","type":"address"}],
        "name": "balanceOf",
        "outputs": [{"name":"","type":"uint256"}],
        "stateMutability": "view",
        "type": "function"
    }
]

weth_contract = w3.eth.contract(address=WETH_ADDRESS, abi=weth_abi)

# Let's wrap 0.01 ETH (for example).
wrap_amount_wei = w3.to_wei(0.01, 'ether')

# Build transaction
wrap_txn = weth_contract.functions.deposit().build_transaction({
    'from': account.address,
    'value': wrap_amount_wei,   # The ETH we are sending
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 200000,
    'gasPrice': w3.eth.gas_price,
    'chainId': 1301
})

# Sign and send
signed_wrap_txn = account.sign_transaction(wrap_txn)
tx_hash = w3.eth.send_raw_transaction(signed_wrap_txn.raw_transaction)
print("Wrapping ETH. Waiting for confirmation...")
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("WETH deposit transaction confirmed! Tx Hash:", receipt.transactionHash.hex())

# Check WETH balance
weth_balance = weth_contract.functions.balanceOf(account.address).call()
print("WETH Balance:", w3.from_wei(weth_balance, 'ether'), "WETH")

Wrapping ETH. Waiting for confirmation...
WETH deposit transaction confirmed! Tx Hash: 26f29beab03b7901200de1a7888c8e0bb8f67ce3129957560fd4c0ef305c434f
WETH Balance: 0.01 WETH


### 7. Approve Uniswap SwapRouter to Spend WETH
We need to let the Uniswap SwapRouter spend our WETH.
From the provided docs, the address for the Uniswap `SwapRouter02`
on Unichain Sepolia is:
```
0xd1AAE39293221B77B0C71fBD6dCb7Ea29Bb5B166
```
We'll set an allowance so that the router can pull our WETH
during the swap.

In [11]:
# 8. Execute Approval Transaction
SWAP_ROUTER_ADDRESS = "0xd1AAE39293221B77B0C71fBD6dCb7Ea29Bb5B166"

# Minimal ERC20 ABI for 'approve' and 'allowance'
erc20_abi = [
    {
        "constant": False,
        "inputs": [
            {
                "name": "_spender",
                "type": "address"
            },
            {
                "name": "_value",
                "type": "uint256"
            }
        ],
        "name": "approve",
        "outputs": [
            {
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [
            {
                "name": "_owner",
                "type": "address"
            },
            {
                "name": "_spender",
                "type": "address"
            }
        ],
        "name": "allowance",
        "outputs": [
            {
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

weth_token_contract = w3.eth.contract(address=WETH_ADDRESS, abi=erc20_abi)
approve_amount = w3.to_wei(1, 'ether')  # Approve 1 WETH (just as an example)

approve_txn = weth_token_contract.functions.approve(
    SWAP_ROUTER_ADDRESS,
    approve_amount
).build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 100000,
    'gasPrice': w3.eth.gas_price,
    'chainId': 1301
})

signed_approve_txn = account.sign_transaction(approve_txn)
tx_hash = w3.eth.send_raw_transaction(signed_approve_txn.raw_transaction)
print("Approving SwapRouter to spend WETH. Waiting for confirmation...")
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Approval transaction confirmed! Tx Hash:", receipt.transactionHash.hex())

# Check allowance
allowance = weth_token_contract.functions.allowance(account.address, SWAP_ROUTER_ADDRESS).call()
print("Current WETH allowance for SwapRouter:", w3.from_wei(allowance, 'ether'), "WETH")

Approving SwapRouter to spend WETH. Waiting for confirmation...
Approval transaction confirmed! Tx Hash: 44b5dae44cc8fb412322a29410e1b4d8379c439b73295f7a758cf4d003b12ccf
Current WETH allowance for SwapRouter: 1 WETH


### 9. Swap WETH for USDC
Now we can use the Uniswap V3 `SwapRouter02` contract's `exactInputSingle()`
to swap from WETH to USDC.

The USDC contract (Circle USDC) on Unichain Sepolia is:
```
0x31d0220469e10c4E71834a79b1f276d740d3768F
```
We will specify:
- `tokenIn` = WETH
- `tokenOut` = USDC
- `fee` = 3000 (0.3%) or 500 (0.05%), depending on available liquidity.
  For demonstration, let's try `3000`.
- `recipient` = our own address.
- `deadline` = a timestamp in the near future.
- `amountIn` = 0.005 WETH (for example).
- `amountOutMinimum` = 0 (for demonstration). Real usage should use slippage checks.


In [12]:
# 10. Build and Send the Swap Transaction

USDC_ADDRESS = "0x31d0220469e10c4E71834a79b1f276d740d3768F"

# Minimal ABI for the SwapRouter function exactInputSingle.
# For a full interface, see: https://github.com/Uniswap/v3-periphery/blob/main/contracts/interfaces/ISwapRouter.sol
swap_router_abi = [
    {
        "name": "exactInputSingle",
        "type": "function",
        "stateMutability": "payable",
        "inputs": [
            {
                "components": [
                    {
                        "internalType": "address",
                        "name": "tokenIn",
                        "type": "address"
                    },
                    {
                        "internalType": "address",
                        "name": "tokenOut",
                        "type": "address"
                    },
                    {
                        "internalType": "uint24",
                        "name": "fee",
                        "type": "uint24"
                    },
                    {
                        "internalType": "address",
                        "name": "recipient",
                        "type": "address"
                    },
                    {
                        "internalType": "uint256",
                        "name": "deadline",
                        "type": "uint256"
                    },
                    {
                        "internalType": "uint256",
                        "name": "amountIn",
                        "type": "uint256"
                    },
                    {
                        "internalType": "uint256",
                        "name": "amountOutMinimum",
                        "type": "uint256"
                    },
                    {
                        "internalType": "uint160",
                        "name": "sqrtPriceLimitX96",
                        "type": "uint160"
                    }
                ],
                "internalType": "struct ISwapRouter.ExactInputSingleParams",
                "name": "params",
                "type": "tuple"
            }
        ],
        "outputs": [
            {
                "internalType": "uint256",
                "name": "amountOut",
                "type": "uint256"
            }
        ]
    }
]

swap_router_contract = w3.eth.contract(address=SWAP_ROUTER_ADDRESS, abi=swap_router_abi)

# We'll swap 0.005 WETH.
amount_in_wei = w3.to_wei(0.005, 'ether')

# Deadline (10 minutes from now). If your local clock is off, adjust accordingly.
import time
deadline = int(time.time()) + 600  # 10 minutes from now

tx = swap_router_contract.functions.exactInputSingle({
    'tokenIn': WETH_ADDRESS,
    'tokenOut': USDC_ADDRESS,
    'fee': 3000,  # 0.3% fee tier
    'recipient': account.address,
    'deadline': deadline,
    'amountIn': amount_in_wei,
    'amountOutMinimum': 0,  # for demo only, set slippage in real use
    'sqrtPriceLimitX96': 0  # no price limit
}).build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 300000,
    'gasPrice': w3.eth.gas_price,
    'chainId': 1301,
    # 'value': 0  # not sending ETH directly, we are swapping WETH
})

signed_tx = account.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
print("Swapping WETH -> USDC. Waiting for confirmation...")
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Swap transaction confirmed! Tx Hash:", receipt.transactionHash.hex())

Swapping WETH -> USDC. Waiting for confirmation...
Swap transaction confirmed! Tx Hash: 3f387cd86c6f349cba1354c13c296685c53180a59af48abe4ad44d2457bd7a6e


### 11. Verify Final Balances
Check how much WETH remains, and how much USDC we received.

In [13]:
# 12. Check WETH and USDC Balances

# Re-use the WETH contract from before, but let's define a USDC contract instance.

usdc_abi = [
    {
        "constant": True,
        "inputs": [{"name":"","type":"address"}],
        "name": "balanceOf",
        "outputs": [{"name":"","type":"uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [
            {
                "name": "",
                "type": "uint8"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

final_weth_balance = weth_contract.functions.balanceOf(account.address).call()
usdc_balance = usdc_contract.functions.balanceOf(account.address).call()

# USDC often has 6 decimals, but let's fetch from the contract to be sure.
usdc_decimals = usdc_contract.functions.decimals().call()

print("Final WETH:", w3.from_wei(final_weth_balance, 'ether'), "WETH")
print("Final USDC:", usdc_balance / (10 ** usdc_decimals), "USDC")

print("\nDone! You have successfully performed a swap on Unichain Sepolia.")

Final WETH: 0.01 WETH
Final USDC: 0.0 USDC

Done! You have successfully performed a swap on Unichain Sepolia.
